In [1]:
import time
'''
Takes a qubitString as input and returns a matrix 
that represents the qubit in the standard basis.
Ex: |00> = [1,0,0,0]
'''
def getQubitVector(bitString):
    bitDict = {}
    bitDict['0'] = np.array([1 , 0])
    bitDict['1'] = np.array([0 , 1])
    matrix = [1]
    
    for c in bitString:
        matrix = np.kron(matrix,bitDict[c])
    return matrix

'''
Given a bitstring, return the decimal number 
represented by the bitstring
'''
def getDecimalNo(bitString):
    val = 0
    n = len(bitString)
    for i in range(0,n):
        val = val + (2**i)*(int)(bitString[-i-1])
    return val

'''
Given a value of n, returns all possible bit-strings of size n.
This function will return a list of 2^n bit strings
'''
def getAllPossibleNBitStrings(n):
    if n==1:
        return ['0','1']
    
    children = getAllPossibleNBitStrings(n-1)
    result = []
    for i in children:
        result.append('0'+i)
        result.append('1'+i)
    return result

In [2]:
import numpy as np

class FunctionObject:
    def __init__(self,fx,n):
        self.__fx = fx
        self.__n = n
        self.__Uf = self.__createUf()
#         self.verifyUf(debug=True)
    
    '''
    Apply the function on the given input.
    '''
    def applyFx(self,input):
        return self.__fx(input)
        
    '''
    The N value which signifies the no. of qubits.
    '''
    def getN(self):
        return self.__n
    
    '''
    The Uf matrix which represents the oracle for this function
    '''
    def getUf(self):
        return self.__Uf
        
    '''
    Using pointer to f(x) and n denoting the no. of qubits,
    it returns an oracle matrix Uf of size 2^(n+1)x2^(n+1) which
    can be used in a Deutsch-Jozsa or Bernstein-Vazirani circuit
    '''
    def __createUf(self):
        Uf = np.zeros((2**(2*self.__n),2**(2*self.__n)))  
#         print("Uf.shape",Uf.shape)
    
        #this dictionary represents the correspondence between bit combinations and Uf indices
        indices_dict = {}
        counter = 0

        inputs = getAllPossibleNBitStrings(self.__n*2)
#         print("self.__n",self.__n)
        for i in inputs:
#             print("")
            #input to the function is the first n bits of the elements (bit patterns) from the dictionary
            x = i[0:self.__n]

            #fx represents the output of function f given the input x
            fx = str(self.applyFx(x))

            #b is the last n bits of the bit patterns from the dictionary
            b = i[self.__n:]
            #below we have the (f(x) + b) mod 2
            bfx = bin(int(b,2)+int(fx,2))[2:]
#             print("init bfx", bfx)
            if(len(bfx)< self.__n):
#                 print("yes less")
                bfx = bfx.zfill(self.__n)
            if(len(bfx)>self.__n):
#                 print("yes more")
                bfx = bfx[-1*self.__n:]
            
#             #the final bit string is the concatenation of the input x and bfx
            result = x + bfx
#             print("x",x,"bfx ", bfx, " result", result, " b", b, "fx", fx)
            #using indices_dict we can now find the index that corresponds to this output
            column = getDecimalNo(result)
            row = getDecimalNo(i)
#             if(column==105):
#             print("row",row,"column ",column,"result ",result," i",i)
#                 print(" bfx", bfx, " b", b, "fx ",fx, "x ", x)
            #now using the target indiex we can create a bit pattern with all 0s and 1 at the target index position
            Uf[row][column] = 1   
        return Uf

In [3]:
import random

class SimonsFunction(FunctionObject):
    
    '''
    Initializes a function object with the passed in fx and ftype values.
    If no fx and ftype values are passed, 
    it will create the function discussed in Waltrous' notes
    '''
    def __init__(self,n,  fntype=None, fx=None, s=None):
        if fx is None:
            functionObj = self.createDefaultFn(n,fntype)            
            fx = functionObj.fx
            self.s = functionObj.getS()
        else:
            self.s = s            

        FunctionObject.__init__(self, fx, n)        
        
    '''
    Creates our defaul function (Waltrous Notes)
    '''    
    def createDefaultFn(self,n,fntype):
        return DefaultFn(n,fntype)
    
    def getS(self):
        return self.s
    

class DefaultFn(SimonsFunction):
    
    def __init__(self,n, fntype):
#         print(n)
        if(n==4):
            self.fn_dict = {'0000':'1010','0001':'1010',
                            '0010':'1100','0011':'1100',
                            '0100':'0000','0101':'0000',
                            '0110':'0101','0111':'0101', 
                            '1000':'0010','1001':'0010',
                            '1010':'1101','1011':'1101',
                            '1100':'1111','1101':'1111',
                            '1110':'0001','1111':'0001'}
            self.s = '0001'
        if(n==3 and fntype==None): #110
            self.fn_dict = {'000':'101','001':'010','010':'000','011':'110','100':'000','101':'110','110':'101','111':'010'}
            self.s = '110'
        if(n==3 and fntype==1): #001
            self.fn_dict = {'000':'101','001':'101','010':'000','011':'000','100':'110','101':'110','110':'010','111':'010'}
            self.s = '001'
        if(n==3 and fntype==2): #010
            self.fn_dict = {'000':'101','001':'010','010':'101','011':'010','100':'000','101':'110','110':'000','111':'110'}
            self.s = '010'
        if(n--3 and fntype==3): #011
            self.fn_dict = {'000':'101','011':'101','001':'111','010':'111','100':'000','111':'000', '101': '011','110':'011' }
            self.s = '011'
        if(n==2):
            self.fn_dict = {'00':'10','01':'10','10':'01','11':'01'}  
            self.s = '01'
        
    def fx(self,inputString):
#         print(self.fn_dict)
        return self.fn_dict[inputString]

In [4]:
from pyquil import Program
from pyquil.gates import *
from pyquil import get_qc
from pyquil.quilatom import unpack_qubit
from pyquil.quil import DefGate
    
def runMainCircuit(functionObj,nTrials,debug=False):
    p = Program()

    UfMatrix = functionObj.getUf()

    #for a n bit function, we need 2n qubits(n ancilla bits)
    n = functionObj.getN()*2
    
    #qc_name = "{}q-qvm".format(n)
    #qc = get_qc(qc_name)
    #qc.compiler.client.timeout = 2000 # number of seconds
    #setting last qubit to 1 - Need to remove this
    #p += X(n-1)
    
    #adding Hadamard gates to all qubits
    
    '''*************************************CHANGE START******************************************'''
    lattice = 'Aspen-4-8Q-A'  # edit as necessary
    qpu = get_qc(lattice)
    #qpu.timeout = 2000
    qubits = qpu.device.qubits()
    print(f'All qubits in this device in order: {qubits}')
    
    reqd_qubits=[]
    
    # includes helper bit due to n+1
    
    for i in range(0,n):
         
        reqd_qubits.append(qubits[i])
    '''**************************************CHANGE OVER******************************************'''
    
    for i in range(0,int(n/2)):
        p += H(reqd_qubits[i]) # CHANGE

    GateName = "UF_GATE"

    #create a gate that uses the Uf matrix and pass all qubits to this as input 
    uf_gate_definition = DefGate(GateName, UfMatrix)
    #qubits = [unpack_qubit(i) for i in range(0,n)]
    
    '''*************************************CHANGE START******************************************'''
    qubits_for_gate = [unpack_qubit(i) for i in reqd_qubits]
    '''**************************************CHANGE OVER******************************************'''

    #adding Uf gate
    #p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits))
    
    '''*************************************CHANGE START******************************************'''
    p+=Program(uf_gate_definition,Gate(name=GateName, params=[],qubits=qubits_for_gate))
    '''**************************************CHANGE OVER******************************************'''

    for i in range(0,int(n/2)):
        p += H(reqd_qubits[i]) #CHANGE
    if debug:
        print(p)
        
    try:
        results = qpu.run_and_measure(p, trials=nTrials)
    except TimeoutError:        
        print("Timeout occured for n: {}".format(n-1))
        return []
        
    return results

In [5]:
import numpy as np

def allZeros(lst):
    allzeros = True
    for l in lst:
        if l!=0:
            allzeros = False
            break
    return allzeros

def xorLists(lst1,lst2):
    result = []
    for l1,l2 in zip(lst1,lst2):
        if l1==l2:
            result.append(0)
        else:
            result.append(1)
    return result

def addRow(vectors, z,columnToMSBOneDict):    
    while allZeros(z) is False:    
        msbOne = z.index(1)
        if msbOne in columnToMSBOneDict.keys():
            otherRow = columnToMSBOneDict[msbOne]
            z = xorLists(vectors[otherRow],z)
        else:
            break
                
    if(allZeros(z)):
        return [],-1
    else:
        msbOne = z.index(1)
        keysList = list(columnToMSBOneDict.keys())
        i = 0
        for i in range(len(keysList)):
            if keysList[i] > msbOne:
                break
            i+=1
        return z,i            
    
def getResultsFromDict(resultsDict,n):
    results = []
    #for i in range(n):
    keys= list(resultsDict.keys())[0:n]
    for i in keys: #!!!!!!!!!!!!!!!!!!!change!!!!!!!!!!!!!!!!!!!!!!!
        results.append(resultsDict[i][0])        
    return results

def addNthVector(vectors,debug=False):
    if debug:
        print("Add nth vector")
        print(vectors)
    newRow = [0 for p in range(len(vectors[0]))]
    lastOne = 0
    for i in range(len(vectors)):
        if vectors[i][i]==1:
            lastOne = i
        else:
            break
            
    newRow[lastOne+1]=1
    vectors.insert(lastOne+1,newRow)
    if debug:
        print(vectors)
    return newRow, vectors


def getS(functionObj,debug=False):
    columnToMSBOneDict = {}
    vectors = []
    
    start_time = time.time()
    callsToQuantumCircuit = 0
    while len(vectors) != functionObj.getN()-1 :
        callsToQuantumCircuit+=1
        results = getResultsFromDict(runMainCircuit(functionObj,1),functionObj.getN())
        if debug:
            print(results)
        z,row = addRow(vectors, results,columnToMSBOneDict)
        if debug:
            print("z","row")
            print(z,row)
        if row is not -1:
            vectors.insert(row,z)
            columnToMSBOneDict[z.index(1)]=row  
    end_time = time.time()
    timeTaken = end_time - start_time
    
    newRow,newVectors = addNthVector(vectors)
    
    if debug:
        print("N Vectors")
        print(newVectors)
        
    a = np.array(newVectors)
    b = np.zeros(functionObj.getN())
    b[newRow.index(1)] = 1
    y = np.linalg.solve(a,b)
    y = [i%2 for i in y]
    x = ''
    for i in range(len(y)):
        x+=str(int(y[i]))
        
    return x,callsToQuantumCircuit,timeTaken

In [6]:
'''
Driver that verifies the validity of the quantum circuits for different F configurations.
'''
class SimonsDriver():
    def __init__(self,n,fnType=None):
        if fnType is None:
            self.functionObj = SimonsFunction(n)
        else:
            self.functionObj = SimonsFunction(n,fnType)
        
    def runAndVerifyQuantumCircuit(self,debug=False):
        print("*"*20)
        x,callsToQuantumCircuit,circuitRunTime = getS(self.functionObj)
        if x is None:
            return
        
        assert x == self.functionObj.getS()
        print("N : {} , QuantumCircuitCalls : {}, S: {}, Time : {}".format(self.functionObj.getN(),callsToQuantumCircuit, x, circuitRunTime))
        print("*"*20)

## Simulation

In [7]:
SimonsDriver(2).runAndVerifyQuantumCircuit()

********************
All qubits in this device in order: [1, 2, 10, 11, 14, 15, 16, 17]
N : 2 , QuantumCircuitCalls : 1, S: 01, Time : 2.4142510890960693
********************


In [9]:
#n = 3
start_time = time.time()
functionObj = SimonsFunction(3)
results = runMainCircuit(functionObj,1)
end_time = time.time()
#print("Results:",results)
print("Time taken", end_time - start_time)

All qubits in this device in order: [1, 2, 10, 11, 14, 15, 16, 17]
Timeout occured for n: 5
Time taken 10.353776216506958


In [10]:
#n=3, type1
start_time = time.time()
functionObj = SimonsFunction(3,1)
results = runMainCircuit(functionObj,1)
end_time = time.time()
#print("Results:",results)
print("Time taken", end_time - start_time)

All qubits in this device in order: [1, 2, 10, 11, 14, 15, 16, 17]
Timeout occured for n: 5
Time taken 10.378262758255005


In [14]:
#n=3, type2
start_time = time.time()
functionObj = SimonsFunction(3,2)
results = runMainCircuit(functionObj,1)
end_time = time.time()
#print("Results:",results)
print("Time taken", end_time - start_time)

All qubits in this device in order: [1, 2, 10, 11, 14, 15, 16, 17]
Timeout occured for n: 5
Time taken 10.379621028900146


In [17]:
#n = 4
start_time = time.time()
functionObj = SimonsFunction(4)
results = runMainCircuit(functionObj,1)
end_time = time.time()
#print("Results:",results)
print("Time taken", end_time - start_time)

All qubits in this device in order: [1, 2, 10, 11, 14, 15, 16, 17]
Timeout occured for n: 7
Time taken 10.802571773529053
